In [ ]:
from __init__ import *

In [ ]:
directory_path = "../output/signal/"

kraft.path.path(directory_path)

In [ ]:
def get_surprise(vector, vector_reference, _0=kraft.CONSTANT.FLOAT_RESOLUTION):

    vector_probability = np.full(vector.shape, _0)

    vector_reference_probability = np.full(vector_reference.shape, _0)

    vector_sum = vector.sum()

    vector_reference_sum = vector_reference.sum()

    if 0 < vector_sum:

        vector_probability += vector / vector_sum

    if 0 < vector_reference_sum:

        vector_reference_probability += vector_reference / vector_reference_sum

    return kraft.information.get_kld(vector_probability, vector_reference_probability)

In [ ]:
def update_surprise(dataframe_number, vector_reference):

    axis_1_size = dataframe_number.shape[1]

    for vector in dataframe_number.to_numpy():

        vector_surprise = get_surprise(vector, vector_reference)

        for i in range(axis_1_size):

            vector[i] = vector_surprise[i]

    kraft.feature_x_sample.summarize(dataframe_number)

In [ ]:
for i, dict_ in enumerate(SETTING["model_data_dicts"]):

    dataframe = pd.read_csv(dict_["file_path"], sep="\t", index_col=0)

    dataframe.columns.name = dict_["axis_1_name"]

    update_surprise(dataframe, dataframe.to_numpy().sum(axis=0))

    negative = dataframe.clip(upper=0)

    positive = dataframe.clip(lower=0)

    negative = negative.loc[
        kraft.series.get_extreme_labels(
            negative.median(axis=1), "<", standard_deviation=2
        ),
        :,
    ]

    positive = positive.loc[
        kraft.series.get_extreme_labels(
            positive.median(axis=1), ">", standard_deviation=2
        ),
        :,
    ]

    negative.index = ("-{}".format(label) for label in negative.index.to_numpy())

    positive.index = ("+{}".format(label) for label in positive.index.to_numpy())

    signal = pd.concat((-negative, positive))

    signal.index.name = dataframe.index.name

    signal.to_csv("{}{}.tsv".format(directory_path, i), sep="\t")